# Advent of Code 2022: Day 22
https://adventofcode.com/2022/day/22


## Part 1
Find the position on a map after executing moves

### Get the data into a list of strings

In [1]:
myfile = open('input.txt', 'r')
data = myfile.read()
data_list = data.split('\n')
# Remove empty value at the bottom of the list.
data_list = data_list[:-1]

### Extract the element that contains the moves

In [2]:
moves = data_list.pop()

### Function to parse the moves into a list

In [3]:
def get_move_list(moves):
  
  # Setup the output list and
  # a temporary string to hold
  # the numbers.
  move_list = []
  temp_str = ""
  
  # Split the moves string into 
  # a list and go through it.
  split = list(moves)
  for i in range(len(split)):
    
    # If the current character is 
    # a digit, add it to the string.
    if split[i].isdigit():
      temp_str += split[i]
      
      # The last move does not contain
      # a direction change, so just 
      # append the number part to the
      # end of the output list. 
      if i == len(split)-1:
        move_list.append([int(temp_str)])
    
    # If a character is not a digit, it
    # is a direction change command, either
    # 'R' or 'L'. Append a list containing
    # the number and direction change to
    # the output list and reset the string.
    else:
      move_list.append([int(temp_str), split[i]])
      temp_str = ""
  
  return move_list

### Function to initialize the board

In [4]:
def setup_board(data):
  
  # Find the longest row in the data
  # to ensure every row on the board
  # will be of the same length.
  max_length = 0
  for i in range(len(data)):
    max_length = max(max_length, len(data[i]))

  # Setup the board as a nested list, with rows equal
  # to the number of rows in the data, and columns equal
  # to the maximum length + 1. The extra padding is done
  # to ensure the final column is empty. 
  board = [[" " for i in range(max_length+1)] for j in range(len(data))]
  
  # Go through the board and mark it with '.' 
  # or '#' according to the data. Also add an
  # empty space to the beginning of each row
  # so that the board will have an empty column
  # to the left. 
  for i in range(len(board)):
    split = data[i]
    for j in range(len(split)):
      board[i][j] = split[j]
    board[i].insert(0, ' ')
  
  # Insert a empty row at the top. Because of how 
  # the data file looks, there should already be
  # an empty row at the bottom. 
  board.insert(0, [' ' for i in range(len(board[0]))])
  return board

### Function to execute a move

In [5]:
def make_move(board, pos, dir, move):
  # Get the numeric part
  # of the move. 
  num = move[0]
  
  # Move right
  if dir == 0:
    for i in range(num):
      
      # Get the position on the board
      # one step to the right. 
      new_pos = board[pos[0]][pos[1]+1]
      
      # Move if possible
      if new_pos == '.':
        pos[1] +=1
      elif new_pos == '#':
        break
      
      # If the new position is empty space
      # this means we must loop around to
      # left side of the board if possible.
      # Get the position of the first dot
      # and first wall in the row (if any)
      # and move if possible.
      elif new_pos == ' ':
        first_dot = len(board[pos[0]])
        fist_wall = len(board[pos[0]])
        if '.' in board[pos[0]]:
          first_dot = board[pos[0]].index('.')
        if '#' in board[pos[0]]:
          first_wall = board[pos[0]].index('#')
        if first_dot < first_wall:
          pos[1] = first_dot
        else:
          break 

  # Move down
  elif dir == 1:
    for i in range(num):
      
      # Get the position on the board
      # one step down.
      new_pos = board[pos[0]+1][pos[1]]
      
      # Move if possible
      if new_pos == '.':
        pos[0] +=1
      elif new_pos == '#':
        break
      
      # If the new position is empty space
      # this means we must loop around to
      # top side of the board if possible.
      # Go from the top until a dot or
      # wall is found, then move if possible.  
      elif new_pos == ' ':
        break_outer = False
        for row in range(len(board)):
          if board[row][pos[1]] == '.':
            pos[0] = row
            break
          if board[row][pos[1]] == '#':
            break
            break_outer = True
        if break_outer:
          break
  
  # Move left
  elif dir == 2:
    for i in range(num):
      
      # Get the position on the board
      # one step to the left. 
      new_pos = board[pos[0]][pos[1]-1]
      
      # Move if possible
      if new_pos == '.':
        pos[1] -=1
      elif new_pos == '#':
        break
      
      # If the new position is empty space
      # this means we must loop around to
      # right side of the board if possible.
      # Get the position of the last dot
      # and last wall in the row (if any)
      # and move if possible.   
      elif new_pos == ' ':
        last_dot = 0
        last_wall = 0
        if '.' in board[pos[0]]:
          last_dot = len(board[pos[0]]) - list(reversed(board[pos[0]])).index('.') -1
        if '#' in board[pos[0]]:
          last_wall = len(board[pos[0]]) - list(reversed(board[pos[0]])).index('#') -1
        if last_dot > last_wall:
          pos[1] = last_dot
        else:
          break 

  # Move up
  elif dir == 3:
    for i in range(num):
      
      # Get the position on the board
      # one step up.
      new_pos = board[pos[0]-1][pos[1]]
      
      # Move if possible.
      if new_pos == '.':
        pos[0] -=1
      elif new_pos == '#':
        break
      
      # If the new position is empty space
      # this means we must loop around to
      # bottom side of the board if possible.
      # Go from the bottom until a dot or
      # wall is found, then move if possible.
      elif new_pos == ' ':
        break_outer = False
        for row in reversed(range(len(board))):
          if board[row][pos[1]] == '.':
            pos[0] = row
            break
          if board[row][pos[1]] == '#':
            break
            break_outer = True
        if break_outer:
          break

  # If the move contains a 
  # direction change, then
  # update the direct.
  if len(move) ==2:
    if move[1] == 'R':
      dir += 1
      dir %= 4
    else:
      dir -=1
      dir %=4
  
  return pos, dir

### Function to get the final position after a number of moves 

In [6]:
def get_final_position(data, moves, part =1):
  
  # Setup the move list, board, starting
  # position and starting direction.
  move_list = get_move_list(moves)
  board = setup_board(data)
  pos = [1,board[1].index('.')]
  dir = 0

  if part == 1:
    for move in move_list:
      pos, dir = make_move(board, pos, dir, move)
  
  if part == 2:
    for move in move_list:
      pos, dir = make_move2(board, pos, dir, move)

  return (1000*pos[0]) + (4*pos[1]) + dir

In [7]:
get_final_position(data_list, moves)

133174

## Part 2
Find the position on a map after executing moves. The board now represents a folded out cube with 50x50 sides with the following pattern:
      
      1 2
      3
    5 4
    6

Solution only works for this specific pattern and for 50x50 sides. 

### Function to execute a move for a cube

In [8]:
def make_move2(board, pos, dir, move):
  num = move[0]
  for i in range(num):
    
    # Move right
    if dir == 0:
      
      # Get the position on the board
      # one step to the right. 
      new_pos = board[pos[0]][pos[1]+1]
      
      # Move if possible
      if new_pos == '.':
        pos[1] +=1
      elif new_pos == '#':
        break
      
      # If the new position is empty space
      # this means we must loop around to
      # another side. Based on the current 
      # position, calculate where to go and
      # move if possible. 
      elif new_pos == ' ':
        if pos[0] >= 1 and pos[0] <=50:
          new_pos = board[151-pos[0]][100]
          if new_pos =='.':
            pos = [151-pos[0], 100]
            dir = 2
          else:
            break
        elif pos[0] >= 51 and pos[0] <=100:
          new_pos = board[50][pos[0]+50]
          if new_pos =='.':
            pos = [50, pos[0]+50]
            dir = 3
          else:
            break
        elif pos[0] >= 101 and pos[0] <=150:
          new_pos = board[151-pos[0]][150]
          if new_pos =='.':
            pos = [151-pos[0], 150]
            dir = 2
          else:
            break
        elif pos[0] >= 151 and pos[0] <=200:
          new_pos = board[150][pos[0]-100]
          if new_pos =='.':
            pos = [150, pos[0]-100]
            dir = 3
          else:
            break
    
    # Move down
    elif dir == 1:

      # Get the position on the board
      # one step down. 
      new_pos = board[pos[0]+1][pos[1]]
      
      # Move if possible
      if new_pos == '.':
        pos[0] +=1
      elif new_pos == '#':
        break
      
      # If the new position is empty space
      # this means we must loop around to
      # another side. Based on the current 
      # position, calculate where to go and
      # move if possible. 
      elif new_pos == ' ':
        if pos[1] >= 101 and pos[1] <=150:
          new_pos = board[pos[1]-50][100]
          if new_pos =='.':
            pos = [pos[1]-50, 100]
            dir = 2
          else:
            break
        elif pos[1] >= 51 and pos[1] <=100:
          new_pos = board[pos[1]+100][50]
          if new_pos =='.':
            pos = [pos[1]+100, 50]
            dir = 2
          else:
            break
        elif pos[1] >= 1 and pos[1] <=50:
          new_pos = board[1][pos[1]+100]
          if new_pos =='.':
            pos = [1, pos[1]+100]
            dir = 1
          else:
            break
    
    # Move left
    elif dir == 2:
      
      # Get the position on the board
      # one step to the left. 
      new_pos = board[pos[0]][pos[1]-1]
      
      # Move if possible
      if new_pos == '.':
        pos[1] -=1
      elif new_pos == '#':
        break
      
      # If the new position is empty space
      # this means we must loop around to
      # another side. Based on the current 
      # position, calculate where to go and
      # move if possible. 
      elif new_pos == ' ':
        if pos[0] >= 1 and pos[0] <=50:
          new_pos = board[151-pos[0]][1]
          if new_pos =='.':
            pos = [151-pos[0],1]
            dir = 0
          else:
            break
        elif pos[0] >= 51 and pos[0] <=100:
          new_pos = board[101][pos[0]-50]
          if new_pos =='.':
            pos = [101, pos[0]-50]
            dir = 1
          else:
            break
        elif pos[0] >= 101 and pos[0] <=150:
          new_pos = board[151-pos[0]][51]
          if new_pos =='.':
            pos = [151-pos[0],51]
            dir = 0
          else:
            break
        elif pos[0] >= 151 and pos[0] <=200:
          new_pos = board[1][pos[0]-100]
          if new_pos =='.':
            pos = [1, pos[0]-100]
            dir = 1
          else:
            break 
    
    # Move up
    elif dir == 3:
      
      # Get the position on the board
      # one step up. 
      new_pos = board[pos[0]-1][pos[1]]
      
      # Move if possible
      if new_pos == '.':
        pos[0] -=1
      elif new_pos == '#':
        break
      
      # If the new position is empty space
      # this means we must loop around to
      # another side. Based on the current 
      # position, calculate where to go and
      # move if possible. 
      elif new_pos == ' ':
        if pos[1] >= 101 and pos[1] <=150:
          new_pos = board[200][pos[1]-100]
          if new_pos =='.':
            pos = [200, pos[1]-100]
            dir = 3
          else:
            break
        elif pos[1] >= 51 and pos[1] <=100:
          new_pos = board[pos[1]+100][1]
          if new_pos =='.':
            pos = [pos[1]+100, 1]
            dir = 0
          else:
            break
        elif pos[1] >= 1 and pos[1] <=50:
          new_pos = board[pos[1]+50][51]
          if new_pos =='.':
            pos = [pos[1]+50, 51]
            dir = 0
          else:
            break

  # If the move contains a 
  # direction change, then
  # update the direct.
  if len(move) ==2:
    if move[1] == 'R':
      dir += 1
      dir %= 4
    else:
      dir -=1
      dir %=4
  
  return pos, dir

In [9]:
get_final_position(data_list, moves, 2)

15410